In [5]:
import numpy as np
import pandas as pd
import warnings

In [6]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem, PandasTools, MACCSkeys, AtomPairs, rdFingerprintGenerator
from rdkit import DataStructs
from rdkit.Chem.rdmolops import PatternFingerprint
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.AtomPairs.Pairs import GetAtomPairFingerprintAsBitVect
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import pearsonr
warnings.filterwarnings("ignore")

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras import layers
from tensorflow.keras import models
from imblearn.over_sampling import SMOTE
import shap
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [8]:
glyco2_final = pd.read_csv('glyco2.csv')

In [9]:
glyco2_final.head()

,Name,SMILES,Concentration (mM),Exp. % MGS
0,4-methoxyphenyl ß-d-galactopyranoside,COc1ccc(O[C@@H]2O[C@H](CO)[C@H](O)[C@H](O)[C@H...,22,78.3
1,4-methoxyphenyl ß-d-glucopyranoside,COc1ccc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@...,22,23.3
2,4-methoxyphenyl α-d-galactopyranoside,COc1ccc(O[C@H]2O[C@H](CO)[C@H](O)[C@H](O)[C@H]...,22,76.4
3,4-methoxyphenyl α-d-glucopyranoside,COc1ccc(O[C@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H...,22,63.2
4,4-bromophenyl ß-d-glucopyranoside,OC[C@H]1O[C@@H](Oc2ccc(Br)cc2)[C@H](O)[C@@H](O...,22,10.0


In [10]:
type(glyco2_final.iloc[(2,1)])

str

In [11]:
df_mf = []
for smile in glyco2_final['SMILES']:
    mol = Chem.MolFromSmiles(smile)
    mf_bitvector = AllChem.GetMorganFingerprintAsBitVect(mol, radius = 1, nBits = 512)
    arr = np.zeros((0,), dtype=np.int32)
    DataStructs.ConvertToNumpyArray(mf_bitvector,arr)
    df_mf.append(arr)

In [12]:
glyco2_final = pd.concat([glyco2_final, pd.DataFrame(df_mf)], axis = 1)

In [13]:
glyco2_final.head()

,Name,SMILES,Concentration (mM),Exp. % MGS,0,1,2,3,4,5,...,502,503,504,505,506,507,508,509,510,511
0,4-methoxyphenyl ß-d-galactopyranoside,COc1ccc(O[C@@H]2O[C@H](CO)[C@H](O)[C@H](O)[C@H...,22,78.3,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,4-methoxyphenyl ß-d-glucopyranoside,COc1ccc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@...,22,23.3,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,4-methoxyphenyl α-d-galactopyranoside,COc1ccc(O[C@H]2O[C@H](CO)[C@H](O)[C@H](O)[C@H]...,22,76.4,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4-methoxyphenyl α-d-glucopyranoside,COc1ccc(O[C@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H...,22,63.2,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4-bromophenyl ß-d-glucopyranoside,OC[C@H]1O[C@@H](Oc2ccc(Br)cc2)[C@H](O)[C@@H](O...,22,10.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [14]:
output_mgs = glyco2_final.iloc[:,3]

In [15]:
type(output_mgs)
output_mgs = np.array(output_mgs)
print(type(output_mgs))

<class 'numpy.ndarray'>


In [16]:
X_train, X_test, y_train, y_test = train_test_split(df_mf, output_mgs, test_size=0.2, random_state=42)


In [17]:
print(type(X_train))
glyco2_final.shape
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

<class 'list'>


In [18]:
# Set random seed
tf.random.set_seed(42)

# Model building
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


In [19]:
model.fit(X_train, y_train, epochs=200, batch_size=8, validation_split=0.2)

Epoch 1/200
18/18 [==============================] - 1s 18ms/step - loss: 2908.2380 - accuracy: 0.0070 - val_loss: 1313.7065 - val_accuracy: 0.0000e+00
Epoch 2/200
18/18 [==============================] - 0s 7ms/step - loss: 1072.2039 - accuracy: 0.0070 - val_loss: 1174.6438 - val_accuracy: 0.0000e+00
Epoch 3/200
18/18 [==============================] - 0s 7ms/step - loss: 825.1801 - accuracy: 0.0070 - val_loss: 1017.5678 - val_accuracy: 0.0000e+00
Epoch 4/200
18/18 [==============================] - 0s 7ms/step - loss: 643.3762 - accuracy: 0.0070 - val_loss: 904.1389 - val_accuracy: 0.0000e+00
Epoch 5/200
18/18 [==============================] - 0s 8ms/step - loss: 541.2489 - accuracy: 0.0070 - val_loss: 914.6407 - val_accuracy: 0.0000e+00
Epoch 6/200
18/18 [==============================] - 0s 7ms/step - loss: 502.8983 - accuracy: 0.0070 - val_loss: 875.5579 - val_accuracy: 0.0000e+00
Epoch 7/200
18/18 [==============================] - 0s 7ms/step - loss: 443.4635 - accuracy: 0.0070

In [20]:
y_pred = model.predict(X_test)

2/2 [==============================] - 0s 2ms/step


In [21]:
glyco2_final.to_csv('glyco2_mf.csv')

In [22]:
y_test

array([64.4,  4.4, 47.1, 87. , 57.4, 12.4, 70.2, 73.6, 50.7, 93.1, 79.2,
       79.7, 76.4, 90.1, 69.7, 73.8, 41.7, 75. , 79.1, 72.2, 68.7, 79.1,
       30.2, 63.8, 79.1, 83.9, 83.8, 88.8, 66.1, 22. , 60.3, 32.8, 81.1,
       67.5, 80. ,  5.9, 66.2, 87.3, 30.7, 18.7, 84.2, 40.9,  4.5, 50.6,
       83.4])

In [23]:
test_loss, test_mae = model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 729.6771 - accuracy: 0.0000e+00


In [24]:
print(test_loss)
print(test_mae)

729.6770629882812
0.0


In [25]:
pcc, _ = pearsonr(y_test, y_pred.flatten())
print(f'Pearson Correlation Coefficient: {pcc}')

Pearson Correlation Coefficient: 0.3642102169283492


In [26]:
r2 = r2_score(y_test, y_pred.flatten())
print(r2)

-0.14382827561155986
